In [4]:
import json
import statistics
from collections import Counter
import os 
import shutil


In [5]:
# Load the JSON file
def load_json(filepath):
    with open(filepath, 'r') as f:
        data = json.load(f)
    return data

# Compute statistics
def compute_keypoint_statistics(data):
    keypoint_counts = [len(model['keypoints']) for model in data]
    
    total_models = len(keypoint_counts)
    total_keypoints = sum(keypoint_counts)
    average_keypoints = statistics.mean(keypoint_counts) if keypoint_counts else 0
    min_keypoints = min(keypoint_counts) if keypoint_counts else 0
    max_keypoints = max(keypoint_counts) if keypoint_counts else 0
    value_counts = Counter(keypoint_counts)
    
    stats = {
        'Total models': total_models,
        'Average keypoints per model': average_keypoints,
        'Minimum keypoints in any model': min_keypoints,
        'Maximum keypoints in any model': max_keypoints,
        'Value counts': value_counts
    }
    
    return stats

In [49]:
filepath = 'knees_train_aug.json' 
data = load_json(filepath)
stats = compute_keypoint_statistics(data)

print("Summary Statistics:")
for k, v in stats.items():
    print(f"{k}: {v}")


Summary Statistics:
Total models: 192
Average keypoints per model: 5
Minimum keypoints in any model: 5
Maximum keypoints in any model: 5
Value counts: Counter({5: 192})


In [46]:
# Paths
mesh_dir = "Guitars/All"  # your source directory containing .ply files
output_dir = "Guitars/9_points/"  # directory to store the filtered files
annotation_file = "guitar.json"  # your annotation json
n = 9

# Create output directory if not exists
os.makedirs(output_dir, exist_ok=True)

# Load annotations
with open(annotation_file, "r") as f:
    annotations = json.load(f)

# Build a lookup for model_ids with exactly 9 keypoints
model_ids_with_n_kp = []
for entry in annotations:
    model_id = entry["model_id"]
    keypoints = entry["keypoints"]
    if len(keypoints) == n:
        model_ids_with_n_kp.append(model_id)

print(f"Found {len(model_ids_with_n_kp)} models with {n} keypoints.")

# Copy matching .ply files to output directory
for model_id in model_ids_with_n_kp:
    src_file = os.path.join(mesh_dir, f"{model_id}.ply")
    dst_file = os.path.join(output_dir, f"{model_id}.ply")
    
    if os.path.exists(src_file):
        shutil.copyfile(src_file, dst_file)
    else:
        print(f"Warning: {src_file} not found!")

print("Done!")


FileNotFoundError: [Errno 2] No such file or directory: 'guitar.json'

In [3]:
# Function that filters and writes filtered JSON data
def filter_and_write_models(filepath, target_count, output_path):
    data = load_json(filepath)
    selected_models = [model for model in data if len(model['keypoints']) == target_count]
    with open(output_path, 'w') as f:
        json.dump(selected_models, f, indent=2)
    print(f"Saved {len(selected_models)} models with {target_count} keypoints to {output_path}")

In [17]:
filter_and_write_models("guitar.json", 9, "guitar_9.json")

Saved 587 models with 9 keypoints to guitar_9.json


### Ground Truth Keypoints for Model

In [38]:
def load_annotations(annotation_file):
    with open(annotation_file, "r") as f:
        annotations = json.load(f)
    
    model_id_to_keypoints = {}
    for entry in annotations:
        model_id = entry["model_id"]
        keypoints = entry["keypoints"]
        model_id_to_keypoints[model_id] = [kp["xyz"] for kp in keypoints]
    
    return model_id_to_keypoints


def get_keypoints_for_model(model_id, annotation_file):
    model_id_to_keypoints = load_annotations(annotation_file)
    
    if model_id in model_id_to_keypoints:
        return model_id_to_keypoints[model_id]
    else:
        print(f"Model ID {model_id} not found in annotations.")
        return None

In [52]:
annotation_file = "knees_flipped.json"  # Path to your annotation file
model_id_example = "12216"
keypoints = get_keypoints_for_model(model_id_example, annotation_file)
if keypoints:
    for i, kp in enumerate(keypoints):
        print(f"{kp[0]:.3f} {kp[1]:.3f} {kp[2]:.3f}")


0.030 -0.027 0.328
-0.027 -0.049 0.295
-0.018 -0.021 0.392
-0.003 0.113 0.296
0.037 -0.196 0.365


### Test Train Split

In [17]:
import random

In [47]:
def load_json(filepath):
    with open(filepath, 'r') as f:
        data = json.load(f)
    return data

def save_json(data, filepath):
    with open(filepath, 'w') as f:
        json.dump(data, f, indent=2)

def split_dataset(filepath, output_train, output_test, train_ratio=0.75, seed=42):
    data = load_json(filepath)
    random.seed(seed)
    random.shuffle(data)
    
    split_index = int(len(data) * train_ratio)
    train_data = data[:split_index]
    test_data = data[split_index:]
    
    save_json(train_data, output_train)
    save_json(test_data, output_test)
    
    print(f"Dataset split completed.")
    print(f"Total samples: {len(data)}")
    print(f"Training samples: {len(train_data)}")
    print(f"Testing samples: {len(test_data)}")



In [48]:
filepath = 'knees_flipped.json' 
split_dataset(filepath, "knees_train.json", "knees_test.json")

Dataset split completed.
Total samples: 43
Training samples: 32
Testing samples: 11


### Flip y and z in json

In [35]:
def flip_yz_axes(input_path, output_path):
    with open(input_path, 'r') as f:
        data = json.load(f)

    for entry in data:
        for kp in entry.get("keypoints", []):
            x, y, z = kp["xyz"]
            kp["xyz"] = [x, z, y]  # Swap Y and Z

    with open(output_path, 'w') as f:
        json.dump(data, f, indent=2)


In [37]:

flip_yz_axes("knees_test.json", "knees_test_flipped.json")